# HW3: 게으른 값계산 작은걸음 의미구조 (PL 2025 Fall @ HNU CE)

- 이름: 이재석
- 학번: 20210492

과제를 하기 전에 위에다 학번과 이름을 꼭 작성하기

----
`CESKz`를 완성하고, 적절한 예제를 작성하여 테스트하라.

테스트 방법은
- `CEKn`에서 중복 계산이 발상하는 식(`Expr`)이 실제로 중복 계산이 발생하는지 `stepCEKn'`로 실행하여 확인하고
- `CESKz`에서는 같은 식에서 중복 계산이 발생하지 않음을 `stepCEKz'`로 실행하여 확인하여 비교하라.

교재에서처럼 `takeWhile`, `not`, `null`, `iterate` 등을 활용하면 좋다.

In [35]:
:opt no-lint
{-# LANGUAGE TypeSynonymInstances FlexibleInstances #-}

In [36]:
type Nm = String -- 변수 이름은 문자열로
data Expr = Var Nm         -- x
          | Lam Nm Expr    -- (λx.e)
          | App Expr Expr  -- (e1 e2)
          deriving (Eq, Ord)

In [37]:
instance Show Expr where -- LaTeX 소스코드 생성하는 Show 인스턴스 직접 선언
  showsPrec _ (Var x) = showString x
  showsPrec p (Lam x e) = showParen (p > 1) $
      showString ("\\lambda "++x++".") . showsPrec 1 e
  showsPrec p (App e1 e2) = showParen (p > 9) $
      showsPrec 9 e1 . showString "\\;" . showsPrec 10 e2
import IHaskell.Display (latex)
dTeX e = display [latex $ "$"++show e++"$"]
instance IHaskellDisplay Expr where display = dTeX

In [38]:
type Env = [(Nm,Clos)] -- 값계산 환경
data Clos = Cl Expr Env deriving (Eq,Ord) -- 클로저

In [39]:
import Data.List (intercalate)
instance {-# OVERLAPS #-} Show Env where
    show env = "\\{"++ intercalate ",\\," (map show env) ++"\\}"
instance {-# OVERLAPS #-} Show (Nm,Clos) where
    show (x,cl) = x++"{\\mapsto}"++show cl
instance Show Clos where
    show (Cl e env) = "\\langle "++show e++","++show env++"\\rangle "

instance {-# OVERLAPS #-} IHaskellDisplay Env  where display = dTeX
instance {-# OVERLAPS #-} IHaskellDisplay (Nm,Clos)
                                               where display = dTeX
instance IHaskellDisplay Clos                  where display = dTeX

In [40]:
type Kont = [(Ctx,Env)] -- 나중에 할 일을 Ctx와 Env의 순서쌍 리스트로 표현
data Ctx = Done | App1 Expr | App2 Expr  deriving (Eq,Ord)

In [41]:
instance Show Ctx where -- LaTeX 소스코드 생성하는 Show 인스턴스 직접 선언
  showsPrec _ Done = showString "\\bullet "
  showsPrec p (App1 e2) = showParen (p > 9) $
      showString "\\bullet \\;" . showsPrec 10 e2
  showsPrec p (App2 e1) = showParen (p > 9) $
      showsPrec 9 e1 . showString "\\; \\bullet"
instance IHaskellDisplay Ctx where display = dTeX

In [42]:
lookupEnv :: Nm -> Env -> [Clos]
lookupEnv x env = case lookup x env of Nothing -> []
                                       Just cl -> [cl]

In [43]:
import Data.List (intercalate)
instance {-# OVERLAPS #-} Show Kont where
  show k = intercalate ",\\," (map show k)
instance {-# OVERLAPS #-} Show (Clos,Kont) where
  show (cl,k) = show cl++";\\,"++show k
instance {-# OVERLAPS #-} Show (Ctx,Env) where
  show (ctx,env) = "\\langle "++show ctx++","++show env++"\\rangle "
instance {-# OVERLAPS #-} IHaskellDisplay Kont where display = dTeX
instance {-# OVERLAPS #-} IHaskellDisplay (Clos,Kont)
                                               where display = dTeX
instance {-# OVERLAPS #-} IHaskellDisplay (Ctx,Env)
                                               where display = dTeX

In [44]:
stepCEKn :: (Clos,Kont) -> [(Clos,Kont)]
stepCEKn (Cl (Var x)     env, k) = [(v,k) | v <- lookupEnv x env]
stepCEKn (Cl (App e1 e2) env, k) = [(Cl e1 env, (App1 e2,env):k)]
stepCEKn (Cl (Lam x e)  env1, (App1 e2,env):k) = [(Cl e ((x,Cl e2 env):env1), k)]
stepCEKn _                       = []

In [45]:
hat f es = concat [f e | e <-es]
stepCEKn' = hat stepCEKn

In [46]:
type Addr = Int
type Env' = [(Nm,Addr)]
data Clos' = Cl' Expr Env' deriving (Eq,Ord)

lookupEnv' :: Nm -> Env' -> [Addr]
lookupEnv' x env = case lookup x env of
                     Nothing -> []
                     Just l -> [l]

In [47]:
import qualified Data.HashMap.Strict as M -- M.funcName으로 활용
import Data.HashMap.Strict (HashMap, (!), (!?)) -- 앞에 M. 없이
type Store = HashMap Int Clos'

In [48]:
import Data.List (intercalate)
instance {-# OVERLAPS #-} Show Env' where
  show env = "\\{"++ intercalate ",\\," (map show env) ++"\\}"
instance {-# OVERLAPS #-} Show (Nm,Addr) where
  show (x,l) = x++"{\\mapsto}"++show l
instance Show Clos' where
  show (Cl' e env) = "\\langle "++show e++","++show env++"\\rangle "
instance {-# OVERLAPS #-} Show (Addr,Clos') where
  show (l,cl) = show l++"{\\mapsto}"++show cl
instance {-# OVERLAPS #-} Show Store where
  show s = "\\{"++ intercalate ",\\," (map show $ M.toList s) ++"\\}"

instance {-# OVERLAPS #-} IHaskellDisplay Env'  where display = dTeX
instance {-# OVERLAPS #-} IHaskellDisplay (Nm,Addr) 
                                                where display = dTeX
instance IHaskellDisplay Clos'                  where display = dTeX
instance {-# OVERLAPS #-} IHaskellDisplay (Addr,Clos')
                                                where display = dTeX
instance {-# OVERLAPS #-} IHaskellDisplay Store where display = dTeX

In [49]:
lookupStore :: Addr -> Store -> [Clos']
lookupStore l st = case st !? l of
                     Nothing -> []
                     Just cl -> [cl]

In [50]:
type Kont' = [Either Addr (Ctx,Env')]

In [51]:
import Data.List (intercalate)
instance {-# OVERLAPS #-} Show Kont' where
  show k = intercalate ",\\," (map showEither k)
         where showEither (Left l)  = show l++"{\\mapsto}\\bullet "
               showEither (Right r) = show r
instance {-# OVERLAPS #-} Show (Clos',Store,Kont') where
  show (cl,s,k) = show cl++";\\,"++show s++";\\,"++show k
instance {-# OVERLAPS #-} Show (Ctx,Env') where
  show (ctx,env) = "\\langle "++show ctx++","++show env++"\\rangle "

instance {-# OVERLAPS #-} IHaskellDisplay Kont' where display = dTeX
instance {-# OVERLAPS #-} IHaskellDisplay (Clos',Store,Kont')
                                                where display = dTeX
instance {-# OVERLAPS #-} IHaskellDisplay (Ctx,Env')
                                                where display = dTeX

In [58]:
-- 새로운 주소 생성: 가장 큰 키 + 1
freshAddr :: Store -> Addr
freshAddr st = if M.null st then 0 else 1 + maximum (M.keys st)

stepCESKz :: (Clos', Store, Kont') -> [(Clos', Store, Kont')]
stepCESKz (Cl' (Var x) env, st, k) =
  [ (cl, st, Left l : k)
  | l <- lookupEnv' x env
  , cl <- lookupStore l st
  ]

-- 함수 호출
stepCESKz (Cl' (App e1 e2) env, st, k) =
  [ (Cl' e1 env, st, Right (App1 e2, env) : k) ]

-- 함수 적용 바인딩
stepCESKz (Cl' (Lam x e) env1, st, Right (App1 e2, env2) : k) =
  let a   = freshAddr st
      st' = M.insert a (Cl' e2 env2) st
      env1' = (x, a) : env1
  in  [ (Cl' e env1', st', k) ]

-- 평가 완료 후 Store 업데이트
stepCESKz (cl@(Cl' (Lam _ _) _), st, Left a : k) =
  let st' = M.insert a cl st
  in  [(cl, st', k)]

-- 종료 상태 없음
stepCESKz _ = []

In [59]:
stepCESKz' = hat stepCESKz -- 여러 번 실행에 편리한 확장된 함수

In [71]:
-- 적절한 식(`Expr`)을 정의
-- (\x. x x) (\y. (u) )
-- 여기서 u는 무한 재귀나 부수효과 없는 단순한 변수라고 가정한 예시
testExpr :: Expr
testExpr = App
             (Lam "x" (App (Var "x") (Var "x")))
             (Lam "y" (Var "u"))


In [82]:
import Data.List (takeWhile)

-- CEKn의 초기 상태
initialCEKn :: [(Clos, Kont)]
initialCEKn = [(Cl testExpr [], [])]

-- CESKz의 초기 상태
initialCESKz :: [(Clos', Store, Kont')]
initialCESKz = [(Cl' testExpr [], M.empty, [])]

-- 실행 트레이스
traceCEKn :: [[(Clos, Kont)]]
traceCEKn = takeWhile (not . null) $ iterate stepCEKn' initialCEKn

traceCESKz :: [[(Clos', Store, Kont')]]
traceCESKz = takeWhile (not . null) $ iterate stepCESKz' initialCESKz

In [83]:
-- stepCEKn'로 적절히 테스트
"==== CEKn Trace ===="
traceCEKn

"==== CEKn Trace ===="

In [84]:
-- stepCESKz'로 적절히 테스트
"==== CESKz Trace ===="
traceCESKz


"==== CESKz Trace ===="